# Trainer Module Example

## Introduction

This jupyter notebook uses the *trainer* module to interact with the Pokemon database. Some of the methods are reviewed and the data is explored using PostgreSQL and Python. There are two classes in the module called PGSQLConnection, which uses your credentials to create a connection object, and TrainerPack, which uses the connection object to open and close the connection when any of its methods are called and require database access. Let's begin with the basic setup.

In [1]:
from trainer import *
import hidden
pd.set_option('display.max_colwidth', 0)
secrets = hidden.secrets()
pgsql_conn = PGSQLConnection(host=secrets['host'], port=secrets['port'], database=secrets['database'],
                             user=secrets['user'], password=secrets['pass'])
tp = TrainerPack(pgsql_connection=pgsql_conn)

Connection opened to PGSQL database.
Connection closed to PGSQL database.



After importing the necessary modules, you create an instance of the PGSQLConnection class, **pgsql_conn**, by passing the database credentials. The **pgsql_conn** variable is then passed to the TrainerPack class to initialize an instance called **tp**. When **tp** is created, it uses the connection once to get some count data from the **trainer** and **trainer_moves** tables. This is why you see the opened/closed connection messages above. 

## TrainerPack Overview

When creating the instance **tp**, the TrainerPack class "passes" the attributes MAX_POKEMON = 6 and MAX_MOVES = 4 by default. The class methods **change_trainer_max** and **change_moves_max** are used to alter those default values. This can be done before or after the instance **tp** is created.

In [2]:
print("Max # of Pokemon in trainer table: " + str(tp.MAX_POKEMON), 
      "\nMax # of moves (per trainer_id) in trainer_moves table: " + str(tp.MAX_MOVES))

Max # of Pokemon in trainer table: 6 
Max # of moves (per trainer_id) in trainer_moves table: 4


In [3]:
TrainerPack.change_trainer_max(3)
TrainerPack.change_moves_max(2)
print("Max # of Pokemon in trainer table: " + str(tp.MAX_POKEMON), 
      "\nMax # of moves (per trainer_id) in trainer_moves table: " + str(tp.MAX_MOVES))

Max # of Pokemon in trainer table: 3 
Max # of moves (per trainer_id) in trainer_moves table: 2


Note that even though the instance **tp** has already been created with the original default values, the class methods above change the max values for the class and the instance. This behavior breaks if you alter the attributes at the instance level.

In [4]:
tp.MAX_POKEMON = 6
tp.MAX_MOVES = 4
TrainerPack.change_trainer_max(1)
TrainerPack.change_moves_max(1)
print("Max # of Pokemon in trainer table: " + str(tp.MAX_POKEMON), 
      "\nMax # of moves (per trainer_id) in trainer_moves table: " + str(tp.MAX_MOVES))
print("Class max # of Pokemon in trainer table: " + str(TrainerPack.MAX_POKEMON), 
      "\nClass max # of moves (per trainer_id) in trainer_moves table: " + str(TrainerPack.MAX_MOVES))

Max # of Pokemon in trainer table: 6 
Max # of moves (per trainer_id) in trainer_moves table: 4
Class max # of Pokemon in trainer table: 1 
Class max # of moves (per trainer_id) in trainer_moves table: 1


Now the "inherited" instance attributes are no longer "linked" to the class attributes.

The count data that was retrieved during the initial setup of **tp** is stored in the attributes **trainer_count** and **moves_count**. These attributes store the current number of Pokmeon in the **trainer** table and the current number of moves per **trainer_id** in the **trainer_moves** table (as a dictionary). The key-value pair (0,0) in the **moves_count** dictionary is an artifact to avoid empty-dictionary errors.

In [5]:
print("# of Pokemon in trainer table: " + str(tp.trainer_count), 
      "\n# of moves for each trainer_id: " + str(tp.moves_count))

# of Pokemon in trainer table: 1 
# of moves for each trainer_id: defaultdict(<class 'int'>, {0: 0, 1: 1})


Let's insert some more Pokemon into the **trainer** table.

In [6]:
tp.insert_trainer([(1, None), (4, None), (7, None), (10, None)])

Connection opened to PGSQL database.
Connection closed to PGSQL database.



You can view the data in the **trainer** table using the **get_select** method. Pass an sql select query as a string to the **get_select** method, and it will return the result as a pandas dataframe. Note that you should pass the query as a raw string by adding the prefix "r" outside the string quotations. This is necessary when passing sql queries through Python since it has its own behavior when reading strings with escape characters, and raw strings allow you to avoid Python's interpretation by sending the sql query "as is" (or "raw") to Postgres.

In [7]:
data = tp.get_select(r"select * from trainer;")
data.columns = ['id', 'poke_id', 'ability_id']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,id,poke_id,ability_id
0,1,65,39.0
1,2,1,NaN
2,3,4,NaN
3,4,7,NaN
4,5,10,NaN


If you try to insert more than one record now, there will be an error and no records will be inserted.

In [8]:
tp.insert_trainer([(13, None), (16, None)])

No values were inserted. Total Pokemon in trainer table would exceed MAX_POKEMON = 6.


The **insert_moves** method has similar functionality, although it will not be used here. Let's conclude the TrainerPack overview by taking a look at the **trainer_moves** table and resetting the **trainer** and **trainer_moves** tables to their original states using the **truncate** method. The **truncate** method removes all rows from a table. When **truncate** is called on the **trainer** table, the **trainer_moves** table is also truncated since it has a foreign key reference to the **trainer** table.

In [9]:
data = tp.get_select(r"select * from trainer_moves;")
data.columns = ['trainer_id', 'move_id']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,trainer_id,move_id
0,1,347


In [10]:
# Reset trainer and trainer_moves table to their original states.
tp.trunc_trainer()
tp.insert_trainer([(65, 39)])
tp.insert_moves([(1, 347)])

Connection opened to PGSQL database.
Connection closed to PGSQL database.

Connection opened to PGSQL database.
Connection closed to PGSQL database.

Connection opened to PGSQL database.
Connection closed to PGSQL database.



## Data Exploration Using PostgreSQL and Python

Let's start our data exploration by examing the **pokedex** table, which contains the basic information for all Pokemon.

In [11]:
data = tp.get_select(r"select * from pokedex order by id;")
data.columns = ['id', 'name', 'ht', 'wt', 'hp', 'at', 'df', 's_at', 's_df', 'speed', 'type', 'evo', 'info']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,id,name,ht,wt,hp,at,df,s_at,s_df,speed,type,evo,info
0,1,bulbasaur,7,69,45,49,49,65,65,45,"[grass, poison]",1,A strange seed was planted on its back at birth. The plant sprouts and grows with this POKéMON.
1,2,ivysaur,10,130,60,62,63,80,80,60,"[grass, poison]",1,"When the bulb on its back grows large, it appears to lose the ability to stand on its hind legs."
2,3,venusaur,20,1000,80,82,83,100,100,80,"[grass, poison]",1,The plant blooms when it is absorbing solar energy. It stays on the move to seek sunlight.
3,4,charmander,6,85,39,52,43,60,50,65,[fire],2,"Obviously prefers hot places. When it rains, steam is said to spout from the tip of its tail."
4,5,charmeleon,11,190,58,64,58,80,65,80,[fire],2,"When it swings its burning tail, it elevates the temperature to unbearably high levels."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,1021,raging-bolt,52,4800,125,73,91,137,89,75,"[electric, dragon]",545,It's said to incinerate everything around it with lightning launched from its fur. Very little is known about this creature.
1021,1022,iron-boulder,15,1625,90,120,80,68,108,124,"[rock, psychic]",547,It resembles a Pokémon described in a dubious magazine as a Terrakion that had been modified by an evil organization.
1022,1023,iron-crown,16,1560,90,72,100,122,108,98,"[steel, psychic]",546,It resembles a mysterious object introduced in a paranormal magazine as a cutting-edge weapon shaped like a Cobalion.
1023,1024,terapagos,2,65,90,65,85,65,85,60,[normal],548,Terapagos protects itself using its power to transform energy into hard crystals. This Pokémon is the source of the Terastal phenomenon.


The **type** column contains each Pokemon's value(s) in a text array. In order to get the counts of the individual types (i.e. ignoring multi-type categories), you can aggregate and count the types after using the **unnest** command.

In [12]:
data = tp.get_select(r"""
select type, count(*) as ct
from (
    select unnest(type) as type
    from pokedex
) as tb
group by type
order by ct desc;
""")
data.columns = ['type', 'count']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,type,count
0,water,154
1,normal,131
2,grass,127
3,flying,109
4,psychic,102
5,bug,92
6,poison,83
7,fire,81
8,ground,75
9,rock,74


This manner of counting the types will result in the total sum exceeding 1025. The counts show that the three most common Pokemon types are water, normal, and grass. You can check the count for the water type using the containment operator on the text array column (without the **unnest** command).

In [13]:
data = tp.get_select(r"select count(*) from pokedex where type @> '{water}';")
data.columns = ['water_type_count']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,water_type_count
0,154


To get the number of Pokemon that are strictly water type, use the equality operator.

In [14]:
data = tp.get_select(r"select count(*) from pokedex where type = '{water}';")
data.columns = ['strict_water_type_count']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,strict_water_type_count
0,74


Note that since arrays are ordered, the equality operator discriminates between permutations of the same values in an array. 

In [15]:
data = tp.get_select(r"select count(*) from pokedex where type = '{water, rock}';")
data.columns = ['water_rock_array_count']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,water_rock_array_count
0,5


In [16]:
data = tp.get_select(r"select count(*) from pokedex where type = '{rock, water}';")
data.columns = ['rock_water_array_count']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,rock_water_array_count
0,6


You can get the count for strictly water-rock type Pokemon, irrespective of type order, using the containment operator in both directions.

In [17]:
data = tp.get_select(r"select count(*) from pokedex where type @> '{water, rock}' and type <@ '{water, rock}';")
data.columns = ['strict_water_rock_type_count']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,strict_water_rock_type_count
0,11


Of course, the code below also produces the same result.

In [18]:
data = tp.get_select(r"select count(*) from pokedex where type @> '{water, rock}';")
data.columns = ['strict_water_rock_type_count']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,strict_water_rock_type_count
0,11


But, this is only because the max cardinality of the arrays in the **type** column is two.

In [19]:
data = tp.get_select(r"select max(cardinality(type)) from pokedex;")
data.columns = ['max_size_of_type_arrays']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,max_size_of_type_arrays
0,2


To get the counts for strict types, including multi-type categories and irrespective of order, use Python to sort the **type** column lists and cast them as tuples. 

In [20]:
data = tp.get_select(r"select type from pokedex;")
data.columns = ['type']
data['type'] = data['type'].apply(lambda x: tuple(sorted(x)))
counts = data.groupby(['type'])['type'].count().sort_values(ascending=False)
print("Strict-Type Counts:\n")
print(counts[counts >= 10].to_string())
print("(... counts < 10 ...)\n\nTotal Sum =", counts.sum())

Connection opened to PGSQL database.
Connection closed to PGSQL database.

Strict-Type Counts:

type
(normal,)           75
(water,)            74
(grass,)            47
(psychic,)          39
(fire,)             36
(electric,)         35
(fighting,)         28
(flying, normal)    27
(bug,)              23
(fairy,)            21
(rock,)             17
(ground,)           17
(ice,)              16
(poison,)           16
(ghost,)            16
(grass, poison)     14
(dragon,)           13
(bug, flying)       13
(dark,)             13
(bug, poison)       12
(rock, water)       11
(steel,)            10
(ghost, grass)      10
(... counts < 10 ...)

Total Sum = 1025


The **evo_set** column in the **pokedex** table contains ids encoding the distinct evolution families. One drawback to this approach of modeling Pokemon evolution is that there is no way to capture the directed graph structure of the evolutionary stages.

In [21]:
data = tp.get_select(r"select count(distinct(evo_set)) from pokedex;")
data.columns = ['distinct_evolution_families']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,distinct_evolution_families
0,541


The original evolution data, stored in the **js_evo** table, can provide additional granularity if you seperate the evolution families into their respective stages. The level 1 stage corresponds to the original/root Pokemon in the evolution graph, and all other levels correspond to the depth in any given evolution chain. It turns out that there is no Pokemon with an evolution chain four levels deep.

In [22]:
data = tp.get_select(r"""
select (body->'id')::int as evo_set, 
        jsonb_path_query_array(body->'chain', '$.species.name'), 
        jsonb_path_query_array(body->'chain', '$.*.species.name'), 
        jsonb_path_query_array(body->'chain', '$.*.*.species.name'),
        jsonb_path_query_array(body->'chain', '$.*.*.*.species.name')
from js_evo
order by evo_set;
""")
data.columns = ['evo_set_id', 'level_1', 'level_2', 'level_3', 'level_4']
data[50:70]

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,evo_set_id,level_1,level_2,level_3,level_4
50,51,[happiny],[chansey],[blissey],[]
51,52,[tangela],[tangrowth],[],[]
52,53,[kangaskhan],[],[],[]
53,54,[horsea],[seadra],[kingdra],[]
54,55,[goldeen],[seaking],[],[]
55,56,[staryu],[starmie],[],[]
56,57,[mime-jr],[mr-mime],[mr-rime],[]
57,58,[scyther],"[scizor, kleavor]",[],[]
58,59,[smoochum],[jynx],[],[]
59,60,[elekid],[electabuzz],[electivire],[]


In [23]:
data[data['level_2'].apply(len) > 1]

,evo_set_id,level_1,level_2,level_3,level_4
21,22,[meowth],"[persian, perrserker]",[],[]
32,33,[slowpoke],"[slowbro, slowking]",[],[]
46,47,[tyrogue],"[hitmonlee, hitmonchan, hitmontop]",[],[]
57,58,[scyther],"[scizor, kleavor]",[],[]
66,67,[eevee],"[vaporeon, jolteon, flareon, espeon, umbreon, leafeon, glaceon, sylveon]",[],[]
95,96,[wooper],"[quagsire, clodsire]",[],[]
108,109,[sneasel],"[weavile, sneasler]",[],[]
134,135,[wurmple],"[silcoon, cascoon]","[beautifly, dustox]",[]
143,144,[nincada],"[ninjask, shedinja]",[],[]
185,186,[snorunt],"[glalie, froslass]",[],[]


In [24]:
data[data['level_3'].apply(len) > 1]

,evo_set_id,level_1,level_2,level_3,level_4
17,18,[oddish],[gloom],"[vileplume, bellossom]",[]
25,26,[poliwag],[poliwhirl],"[poliwrath, politoed]",[]
134,135,[wurmple],"[silcoon, cascoon]","[beautifly, dustox]",[]
139,140,[ralts],[kirlia],"[gardevoir, gallade]",[]
404,413,[cosmog],[cosmoem],"[solgaleo, lunala]",[]


In [25]:
data[(data['level_2'].apply(len) > 1) & (data['level_3'].apply(len) > 1)]

,evo_set_id,level_1,level_2,level_3,level_4
134,135,[wurmple],"[silcoon, cascoon]","[beautifly, dustox]",[]


Wurmple and Goomy are the only Pokémon with a branched evolution whose evolutions can also evolve again. But, Goomy's evolution chains are essentially the same except in one chain the evolutions are all kind of fat, and therefore, the names are identical. So, you will not see it appear in the results above.

In [26]:
data[data['level_1'].apply(lambda x: x == ['goomy'])]

,evo_set_id,level_1,level_2,level_3,level_4
353,362,[goomy],[sliggoo],[goodra],[]


Suppose you wish to get all the Pokemon ranked in the top six for the least amount of moves available (ties included). Simply join the **pokedex** and **pokemon_moves** tables and rank over the aggregate counts. Note that you are able to rank over the aggregate counts with a single select statment since window functions are executed after the "group by" command. 

In [27]:
data = tp.get_select(r"""
with cte as (
    select pd.id, pd.name, count(*), rank() over(order by count(*)) as rk
    from pokedex as pd
    left join pokemon_moves as pm
        on pd.id = pm.poke_id
    group by pd.id
)
select * from cte where rk <= 6 order by rk, id;
""")
data.columns = ['id', 'name', 'move_count', 'ranking']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,id,name,move_count,ranking
0,132,ditto,1,1
1,201,unown,1,1
2,235,smeargle,1,1
3,789,cosmog,2,4
4,790,cosmoem,2,4
5,13,weedle,4,6


Join the **pokedex**, **pokemon_moves**, **moves**, and **types** tables to get more information for the Pokemon listed above. 

In [28]:
data = tp.get_select(r"""
select pd.id, pd.name, mv.id, mv.name, mv.pp, mv.damage::int, mv.accuracy::numeric, ty.name, mv.info
from pokedex as pd
left join pokemon_moves as pm
    on pd.id = pm.poke_id
left join moves as mv
    on mv.id = pm.move_id
left join types as ty
    on ty.id = mv.type
where pd.id = any('{132, 201, 235, 789, 790, 13}')
order by pd.id;
""")
data.columns = ['pk_id', 'pk_name', 'mv_id', 'mv_name', 'points', 'damage', 'accuracy', 'type', 'info']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,pk_id,pk_name,mv_id,mv_name,points,damage,accuracy,type,info
0,13,weedle,40,poison-sting,35,15.0,100,poison,An attack that may poison the target.
1,13,weedle,81,string-shot,40,NaN,95,bug,A move that lowers the foe's SPEED.
2,13,weedle,450,bug-bite,20,60.0,100,bug,"The user bites the foe. If the foe is holding a Berry, the user eats it and gains its effect."
3,13,weedle,527,electroweb,15,55.0,95,electric,The user captures and attacks opposing Pokémon by using an electric net. It reduces the targets’ Speed stat.
4,132,ditto,144,transform,10,NaN,None,normal,The user assumes the foe's guise.
5,201,unown,237,hidden-power,15,60.0,100,normal,The power varies with the POKéMON.
6,235,smeargle,166,sketch,1,NaN,None,normal,Copies the foe's move permanently.
7,789,cosmog,100,teleport,20,NaN,None,psychic,A move for fleeing from battle.
8,789,cosmog,150,splash,40,NaN,None,normal,Has no effect whatsoever.
9,790,cosmoem,100,teleport,20,NaN,None,psychic,A move for fleeing from battle.


Finally, let's use "to_tsquery" and "to_tsvector" to perform some text-based search queries. If you want to find all the Pokemon abilities that contain either the word "energy" or the word "sound" (or both) in their description, along with the number of Pokemon that have those abilities, then you can filter the **abilities** table using "to_tsquery" and "to_tsvector" in the "where" clause, and finish the query by joining the  **pokemon_abilities** table and aggregating.

In [29]:
data = tp.get_select(r"""
select ab.id, ab.name, ab.info, count(*) as ct
from abilities as ab
left join pokemon_abilities as pa
    on ab.id = pa.ability_id
where to_tsquery('english', 'energy | sound') @@ to_tsvector('english', ab.info)
group by ab.id
order by ct, ab.id;
""")
data.columns = ['id', 'name', 'info', 'pk_count']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,id,name,info,pk_count
0,244,punk-rock,Boosts the power of sound-based moves. The Pokémon also takes half the damage from these kinds of moves.,1
1,304,tera-shift,"When the Pokémon enters a battle, it absorbs the energy around itself and transforms into its Terastal Form.",1
2,204,liquid-voice,All sound-based moves become Water-type moves.,3
3,281,protosynthesis,Boosts the Pokémon's most proficient stat in harsh sunlight or if the Pokémon is holding Booster Energy.,10
4,282,quark-drive,Boosts the Pokémon's most proficient stat on Electric Terrain or if the Pokémon is holding Booster Energy.,10
5,43,soundproof,Avoids sound-based moves.,15


Note that the cleaning process performed on the information data removed certain special characters from the text. Unfortunately, those characters were sometimes located in the middle of words, and this resulted in some typos. Although the query below is able to capture certain typos in the results, in general, there may be some difficulty in locating records with text-based search if there are typos. For example, searching the **pokedex** table **info** column for the word "contains" instead of "swimming" will not return the record for Pokemon id 183, which is shown below in the original results.

In [30]:
data = tp.get_select(r"""
select * from pokedex 
where to_tsquery('english', 'swimming') @@ to_tsvector('english', info)
order by id desc limit 11;
""")
data.columns = ['id', 'name', 'ht', 'wt', 'hp', 'at', 'df', 's_at', 's_df', 'speed', 'type', 'evo', 'info']
data

Connection opened to PGSQL database.
Connection closed to PGSQL database.



,id,name,ht,wt,hp,at,df,s_at,s_df,speed,type,evo,info
0,1004,chi-yu,4,49,55,80,80,135,120,100,"[dark, fire]",532,"It controls flames burning at over 5,400 degrees Fahrenheit. It casually swims through the sea of lava it creates by melting rock and sand."
1,614,beartic,26,2600,95,130,80,70,80,50,[ice],310,"It can make its breath freeze at will. Very able in the water, it swims around in northern seas and catches prey."
2,580,ducklett,5,55,62,44,50,44,50,55,"[water, flying]",295,These bird Pokémon are excellent divers. They swim around in the water eating their favorite food--peat moss.
3,490,manaphy,3,14,100,100,100,100,100,100,[water],250,"Born on a cold seafloor, it will swim great distances to return to its birthplace."
4,370,luvdisc,6,87,43,30,55,40,65,97,[water],190,LUVDISC live in shallow seas in the tropics. This heart-shaped POKéMON earned its name by swimming after loving couples it spotted in the ocean’s waves.
5,365,walrein,14,1506,110,80,90,95,90,65,"[ice, water]",187,It swims through icy seas while shattering ice floes with its large tusks. It is protected by its thick blubber.
6,321,wailord,145,3980,170,90,45,90,45,60,[water],161,"WAILORD is the largest of all identified POKéMON up to now. This giant POKéMON swims languorously in the vast open sea, eating massive amounts of food at once with its enormous mouth."
7,226,mantine,21,2200,85,40,70,80,140,70,"[water, flying]",116,"As it majestically swims, it doesn't care if REMORAID attach to it for scavenging its leftovers."
8,195,quagsire,14,750,95,85,85,65,65,35,"[water, ground]",96,"This carefree POKéMON has an easy-going nature. While swimming, it always bumps into boat hulls."
9,183,marill,4,85,70,20,50,20,50,40,"[water, fairy]",90,"The tip of its tail, which con­ tains oil that is lighter than wa­ ter, lets it swim without drowning."
